In [1]:
import pandas as pd

### Original Training data has been filtered into Toxic/Non-Toxic Samples. Only Toxic Samples are being used for Paraphrase Generation using T5

In [2]:
df2 = pd.read_csv("toxic_data.csv")

In [3]:
df2.drop("Unnamed: 0", axis=1, inplace=True)

In [4]:
df2.head()

,text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,cock suck before you piss around on my work,1,1,1,0,1,0
1,hey what is it talk what is it an exclusive gr...,1,0,0,0,0,0
2,bye don t look come or think of comming back t...,1,0,0,0,0,0
3,you are gay or antisemmitian archangel white t...,1,0,1,0,1,1
4,fuck your filthy mother in the ass dry,1,0,1,0,1,0


### Google's T5 Pretrained Model is used for Paraphrase Generation

In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("hetpandya/t5-small-tapaco")
model = T5ForConditionalGeneration.from_pretrained("hetpandya/t5-small-tapaco")

In [13]:
def get_paraphrases(sentence, prefix="paraphrase: ", n_predictions=4, top_k=120, max_length=200,device="cpu"):
        
        text = prefix + sentence + " </s>"
        encoding = tokenizer.encode_plus(text, padding=True, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

        model_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_masks,
            do_sample=True,
            max_length=max_length,
            top_k=top_k,
            top_p=0.98,
            early_stopping=True,
            num_return_sequences=n_predictions,
        )

        outputs = []
        for output in model_output:
            generated_sent = tokenizer.decode(
                output, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            if (
                generated_sent.lower() != sentence.lower()
                and generated_sent not in outputs
            ):
                outputs.append(generated_sent)
        return outputs

### Severe Toxic, Threat and Identity Hate Categories are having less samples compared to other categories. Paraphrases are generated only for samples of these categories

In [7]:
# severe_toxic = df2.loc[(df2["severe_toxic"]==1)][:100]
# threat = df2.loc[(df2["threat"]==1)]
identity_hate = df2.loc[(df2["identity_hate"]==1)][:200]

In [8]:
identity_hate

,text,toxic,severe_toxic,obscene,threat,insult,identity_hate
3,you are gay or antisemmitian archangel white t...,1,0,1,0,1,1
14,a pair of jew hating weiner nazi schmucks,1,0,1,0,1,1
18,i think that your a fagget get a oife and burn...,1,0,1,1,1,1
24,kill all nigger s i have hard that others have...,1,0,1,0,1,1
26,u r a tw fuck off u gay boy u r smelly fuck ur...,1,0,1,0,1,1
...,...,...,...,...,...,...,...
2246,bahamut is gay it s true you little bitch,1,0,1,0,1,1
2265,i am a fucking retard ed nigger,1,0,1,0,1,1
2267,get a life you animal fucker shut up you fucki...,1,1,1,0,1,1
2277,fuck you you fucking nigger bag of shit i hope...,1,1,1,1,1,1


In [9]:
identity_hate.reset_index(inplace=True)

In [10]:
identity_hate.drop("index", axis=1, inplace=True)

### For every sample, 4 paraphrases are generated and are appended to the dataframe

In [11]:
def augment(df, paraphrases, n):
    for j, sent in enumerate(paraphrases):
        num = n + j
        df.at[num, 'text'] = sent
        df.at[num, feat] = cols
    return df

In [ ]:
df_feature = identity_hate
feat = identity_hate.columns.to_list()[1:]

for i in range(len(df_feature)):
    sentence = df_feature.loc[i]["text"]
    cols = df_feature.loc[i][feat]
    paraphrases = get_paraphrases(sentence)
    u_len = len(identity_hate)
    out = augment(identity_hate, paraphrases, u_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (3215 > 512). Running this sequence through the model will result in indexing errors


### Original and augmented samples are finally saved into a csv file for further processing

In [ ]:
out.to_csv("identity_hate_generated.csv")